<a href="https://colab.research.google.com/github/Ariara61/ESAA/blob/main/%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%99%84%EB%B2%BD%EA%B0%80%EC%9D%B4%EB%93%9C_487~496.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##04 텍스트 분류 실습 - 20 뉴스그룹 분류
* 텍스트 분류: 특정 문서의 분류를 학습 데이터를 통해 학습해 모델을 생성한 뒤 이 학습 모델을 이용해 다른 문서의 분류를 예측하는 것
* fetch_20newsgroups()를 이용해 뉴스그룹의 분류를 수행해 볼 수 있는 예제 데이터를 제공
* 텍스트를 피처 벡터화로 변환하면 일반적으로 희소 행렬 형태가 됨.
* 이러한 희소 행렬에 분류를 효과적으로 잘 처리할 수 있는 알고리즘은 로지스틱 회귀, 선형 서포트 벡터 머신, 나이브 베이지 등
* 텍스트를 기반으로 분류를 수행할 때는 먼저 텍스트를 정규화된 뒤 피처 벡터화를 적용. 이후 적합한 머신러닝 알고리즘을 적용해 분류를 학습/예측/평가.

**텍스트 정규화**
* fetch_20newsgroups()는 인터넷에서 로컬 컴퓨터로 데이터를 먼저 내려받은 후에 메모리로 데이터를 로딩함.

In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset = "all", random_state = 156)

In [2]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


* filenames: fetch_20newsgruops() API가 인터넷에서 내려받아 로컬 컴퓨터에 저장하는 디렉터리와 파일명을 지칭

In [3]:
import pandas as pd

print("target 클래스의 값과 분포도 \n", pd.Series(news_data.target).value_counts().sort_index())
print("target 클래스의 이름들 \n", news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [4]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

* 텍스트 데이터를 확인해 보면 뉴스그룹 기사의 내용뿐만 아니라 뉴스그룹 제목, 작성자, 소속, 이메일 등의 다양한 정보를 가지고 있음.
* 이 중에서 내용을 제외하고 제목 등의 다른 정보는 제거. 왜냐하면 제목과 소속, 이메일 주소 등의 헤더와 푸터 정보들은 뉴스그룹 분류의 Target 클래스 값과 유사한 데이터를 가지고 있는 경우가 많기 때문. 이 피처들을 포함하게 되면 왠만한 ML 알고리즘을 적용해도 상당히 높은 예측 성능을 나타냄.
* remove 파라미터를 이용하면 뉴스그룹 기사의 헤더, 푸터 등을 제거할 수 있음.
* fetch_20newsgroups()는 subset 파라미터를 이용해 학습 데이터 세트와 테스트 데이터 세트를 분리해 내려받을 수 있음

In [5]:
from sklearn.datasets import fetch_20newsgroups

# subset = "train"으로 학습요 데이터만 추출, remove = ("headers", "footers", "quotes")f로 내용만 추출
train_news = fetch_20newsgroups(subset = "train", remove = ("headers", "footers", "quoters"),
                   random_state = 156)
X_train = train_news.data
y_train = train_news.target

# subset = "test"으로 테스트 데이터만 추출, remove = ("headers", "footers", "quotes")로 내용만 추출
test_news = fetch_20newsgroups(subset = "test", remove = ("headers", "footers", "quotes"),
                              random_state = 156)
X_test = test_news.data
y_test = test_news.target
print("학습 데이터 크기 {0}, 테스트 데이터 크기 {1}".format(len(train_news.data),
                                             len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


**피처 벡터화 변환과 머신러닝 모델 학습/예측/평가**
* CountVectorizer를 이용해 학습 데이터의 텍스트를 피처 벡터화
* 테스트 데이터에 CountVectorizer를 적용할 때는 반드시 학습 데이터를 이용해 fit()이 수행된 CountVectorizer 객체를 이용해 테스트 데이터를 변환해야 함. 그래야만 학습 시 설정된 CountVectorizer의 피처 개수와 테스트 데이터를 CountVectorizer로 변환할 피처 개수가 같아짐.
* 테스트 데이터의 피처 벡터화는 학습 데이터에 사용된 CountVectorizer 객체 변수인 cnt_vect.transform()을 이용해변환.
* 테스트 데이터의 피처 벡터화 시 fit_transform()을 사용하면 안 됨.
* CountVectorizer.fit_transform(테스트 데이터)을 테스트 데이터 세트에 적용하면 테스트 데이터 기반으로 다시 CountVectorizer가 fit()을 수행하고 transform()하기 때문에 학습 시 사용된 피처 개수와 예측 시 사용할 피처 개수가 달라짐.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorizer으로 피처 벡터화 변환 수행.
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피처 벡터화 변환 수행.
X_test_cnt_vect = cnt_vect.transform(X_test)

print("학습 데이터 텍스트의 CountVectorizer Shape: ", X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape:  (11314, 114751)


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용해 학습/예측/평가 수행.
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print("CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test,
                                                                                   pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.573


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환.
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용해 학습/예측/평가 수행.
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print("TF-IDF Logistic Regression의 예측 정확도는 {:.3f}".format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.663


* TF-IDF가 단순 카운트 기반보다 훨씬 높은 예측 정확도를 제공
* 일반적으로 문서 내에 텍스트가 많고 많은 문서를 가지는 텍스트 분석에서 카운트 벡터화보다는 TF-IDF 벡터화가 좋은 예측 결과를 도출
* 텍스트 분석에서 머신러닝 모델의 성능을 향상시키는 중요한 2가지 방법은 최적의 ML 알고리즘을 선택하는 것과 최상의 피처 전처리를 수행하는 것
* 텍스트 정규화나 Count\TF-IDF 기반 피처 벡터화를 어떻게 효과적으로 적용했는지가 텍스트 기반의 머신러닝 성능에 큰 영향을 미칠 수 있음.

In [12]:
# stop words 필터링을 추가하고 ngram을 기본 (1, 1)에서 (1, 2)로 변경해 피처 벡터화 적용.
tfidf_vect = TfidfVectorizer(stop_words = "english", ngram_range = (1, 2), max_df = 300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print("TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test, pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.683


In [13]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행. CV는 3폴드 세트로 설정.
params = {"C": [0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid = params, cv = 3, scoring = "accuracy", verbose = 1)
grid_cv_lr.fit(X_train_tfidf_vect, y_train)
print("Logistic Regression best C parameter: ", grid_cv_lr.best_params_)

# 최적 C 값으로 학습된 grid_cv로 예측 및 정확도 평가.
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print("TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Logistic Regression best C parameter:  {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.689


* 로지스틱 회귀의 C가 10일 때 GridSearchCV의 교차 검증 테스트 세트에서 가장 좋은 예측 성능을 나타냈으며, 이를 테스트 데이터 세트에 적용해 약 0.703으로 이전보다 약간 향상된 성능 수치가 됨

**사이킷런 파이프라인(Pipeline) 사용 및 GridSearchCV와의 결합**
* 사이킷런의 Pipeline 클래스를 이용하면 피처 벡터화와 ML 알고리즘 학습/예측을 위한 코드 작성을 한 번에 진행할 수 있음
* 데이터 전처리와 머신러닝 학습 과정을 통일된 API 기반에서 처리할 수 있어 더 직관적인 ML 모델 코드를 생성할 수 있으며, 대용량 데이터의 피처 벡터화 결과를 별도 데이터로 저장하지 않고 스트림 기반에서 바로 머신러닝 알고리즘의 데이터로 입력할 수 있기 때문에 수행 시간을 절약할 수 있음.
* 사이킷런 파이프라인은 텍스트 기반의 피처 벡터화뿐만 아니라 모든 데이터 전처리 작업과 Estimator를 결합할 수 있음.

In [15]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([("tfidf_vect", TfidfVectorizer(stop_words = "english")),
                     ("lr_clf", LogisticRegression(random_state = 156))])

* TfidVectorizer 객체를 tfidf_vect라는 객체 변수 명으로, LogisticRegression 객체를 lr_clf라는 객체 변수 명으로 생성한 뒤 이 두 개의 객체를 파이프라인으로 연결하는 Pipeline 객체 pipeline을 생성한다는 의미
* Pipeline 방식을 적용하면 머신러닝 코드를 더 직관적이고 쉽게 작성할 수 있음.

In [16]:
from sklearn.pipeline import Pipeline

# TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성
pipeline = Pipeline([
    ("tfidf_vect", TfidfVectorizer(stop_words = "english", ngram_range = (1, 2), max_df = 300)),
    ("lr_clf", LogisticRegression(C = 10))
])

# 별도의 TfidVectorizer 객체의 fit(), transform()과 LogisticRegression의 fit(), predict()가
# 필요 없음.
# pipeline의 fit()과 predict()만으로 한꺼번에 피처 벡터화와 ML 학습/예측이 가능.
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print("Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}".format(
    accuracy_score(y_test, pred)
))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline을 통한 Logistic Regression의 예측 정확도는 0.689


* 사이킷런은 GridSearchCV 클래스의 생성 파라미터로 Pipeline을 입력해 Pipeline 기반에서도 하이퍼 파라미터 튜닝을 GridSearchCV 방식으로 진행할 수 있게 함.
* 이렇게 하면 피처 벡터화를 위한 파라미터와 ML 알고리즘의 하이퍼 파라미터를 모두 한 번에 GridSearchCV를 이용해 최적화할 수 있음.
* GridSearchCV에 Estimator가 아닌 Pipeline을 입력할 경우에는 key 값이 하이퍼 파라미터명이 객체 변수명과 결합돼 제공됨.
* Pipeline + GridSearchCV를 적용할 때 유의할 점은 모두의 파라미터를 최적화하려면 너무 많은 튜닝 시간이 소모된다는 점

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ("tfidf_vect", TfidfVectorizer(stop_words = "english")),
    ("lr_clf", LogisticRegression())
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(_) 2개를 연달아 붙여 GridSearchCV에 사용될
# 파라미터/하이퍼 파라미터 이름과 값을 설정.
params = {"tfidf_vect__ngram_range": [(1, 1), (1, 2), (1, 3)],
          "tfidf_vect__max_df": [100, 300, 700],
          "lr_clf__C": [1, 5, 10]
          }

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid = params, cv = 3, scoring = "accuracy", verbose = 1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print("Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}".format(
    accuracy_score(y_test, pred)
))

Fitting 3 folds for each of 27 candidates, totalling 81 fits


* max_df 파라미터가 700, ngram_range 파라미터가 (1, 2)로 피처 벡터화된 데이터 세트에 LogisticRegression의 C 하이퍼 파라미터에 10을 적용해 예측 분류를 수행할 때 가장 좋은 검증 세트 성능 수치가 도출됨.
* 로지스틱 회귀 외에 서포 벡터머신과 나이브 베이즈 알고리즘오 희소 행렬 기반의 텍스트 분류에 자주 사용되는 머신러닝 알고리즘.